In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive")

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("SampleLabeledArticles_Transformers.csv", low_memory=False)

In [5]:
data.head()

,Alternate URL,Article Abstract,Author Biography,Banner,Biographical Categories,Body,Byline,Column Name,Column Number,Correction Date,Correction Text,Credit,Dateline,Day Of Week,Descriptors,Feature Page,General Online Descriptors,Guid,Headline,Kicker,Lead Paragraph,Locations,Names,News Desk,Normalized Byline,Online Descriptors,Online Headline,Online Lead Paragraph,Online Locations,Online Organizations,Online People,Online Section,Online Titles,Organizations,Page,People,Publication Date,Publication Day Of Month,Publication Month,Publication Year,Section,Series Name,Slug,Taxonomic Classifiers,Titles,Types Of Material,Url,Word Count,Filename,Sentiment and score,Sentiment,Score
0,NaN,NaN,NaN,NaN,NaN,LEAD: *3*** COMPANY REPORTS ** *3*AAR CORP (NY...,NaN,NaN,3,NaN,NaN,The New York Times,NaN,Thursday,COMPANY REPORTS,NaN,NaN,0,AAR CORP reports earnings for Qtr to Nov 30,NaN,LEAD: *3*** COMPANY REPORTS ** *3*AAR CORP (NY...,NaN,NaN,Financial Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business,NaN,NaN,44,NaN,19870101T000000,1,1,1987,1,NaN,NaN,NaN,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,107,C:\\Users\\gsood\\Dropbox\\data\\nyt\\nytextra...,"('NEGATIVE', 0.9835587739944458)",NEGATIVE,0.983559
1,NaN,NaN,NaN,NaN,NaN,LEAD: *3*** COMPANY REPORTS ** *3*AMERICAN CYT...,NaN,NaN,3,NaN,NaN,The New York Times,NaN,Thursday,COMPANY REPORTS,NaN,NaN,1,AMERICAN CYTOGENETICS reports earnings for Qtr...,NaN,LEAD: *3*** COMPANY REPORTS ** *3*AMERICAN CYT...,NaN,NaN,Financial Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business,NaN,NaN,44,NaN,19870101T000000,1,1,1987,1,NaN,NaN,NaN,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,52,C:\\Users\\gsood\\Dropbox\\data\\nyt\\nytextra...,"('NEGATIVE', 0.984883189201355)",NEGATIVE,0.984883
2,NaN,NaN,NaN,NaN,NaN,LEAD: *3*** COMPANY REPORTS ** *3*APPLIED POWE...,NaN,NaN,3,NaN,NaN,The New York Times,NaN,Thursday,COMPANY REPORTS,NaN,NaN,2,APPLIED POWER reports earnings for Qtr to Nov 30,NaN,LEAD: *3*** COMPANY REPORTS ** *3*APPLIED POWE...,NaN,NaN,Financial Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business,NaN,NaN,44,NaN,19870101T000000,1,1,1987,1,NaN,NaN,NaN,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,39,C:\\Users\\gsood\\Dropbox\\data\\nyt\\nytextra...,"('NEGATIVE', 0.9841152429580688)",NEGATIVE,0.984115
3,NaN,NaN,NaN,NaN,NaN,LEAD: *3*** COMPANY REPORTS ** *3*BARRETT RESO...,NaN,NaN,3,NaN,NaN,The New York Times,NaN,Thursday,COMPANY REPORTS,NaN,NaN,3,BARRETT RESOURCES reports earnings for Year to...,NaN,LEAD: *3*** COMPANY REPORTS ** *3*BARRETT RESO...,NaN,NaN,Financial Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business,NaN,NaN,44,NaN,19870101T000000,1,1,1987,1,NaN,NaN,NaN,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,56,C:\\Users\\gsood\\Dropbox\\data\\nyt\\nytextra...,"('POSITIVE', 0.6753167510032654)",POSITIVE,0.675317
4,NaN,NaN,NaN,NaN,NaN,LEAD: *3*** COMPANY REPORTS ** *3*CANAVERAL IN...,NaN,NaN,3,NaN,NaN,The New York Times,NaN,Thursday,COMPANY REPORTS,NaN,NaN,4,CANAVERAL INTERNATIONAL reports earnings for Y...,NaN,LEAD: *3*** COMPANY REPORTS ** *3*CANAVERAL IN...,NaN,NaN,Financial Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business,NaN,NaN,44,NaN,19870101T000000,1,1,1987,1,NaN,NaN,NaN,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,54,C:\\Users\\gsood\\Dropbox\\data\\nyt\\nytextra...,"('NEGATIVE', 0.9052642583847046)",NEGATIVE,0.905264


In [6]:
X = data["Body"]
y = data["Sentiment"]

In [7]:
X.head()

0    LEAD: *3*** COMPANY REPORTS ** *3*AAR CORP (NY...
1    LEAD: *3*** COMPANY REPORTS ** *3*AMERICAN CYT...
2    LEAD: *3*** COMPANY REPORTS ** *3*APPLIED POWE...
3    LEAD: *3*** COMPANY REPORTS ** *3*BARRETT RESO...
4    LEAD: *3*** COMPANY REPORTS ** *3*CANAVERAL IN...
Name: Body, dtype: object

In [8]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer()
word_count_vector = cv.fit_transform(X)
print(word_count_vector.shape)

(2000, 15048)


In [10]:
tokens = cv.get_feature_names()
print(tokens)

['00', '000', '0003', '0004', '002', '0025', '0033', '004', '006', '008', '009', '01', '010', '011', '013', '014', '015', '016', '017', '019', '02', '020', '021', '022', '023', '024', '025', '027', '0275', '028', '029', '03', '032', '038', '039', '04', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '05', '050', '051', '052', '054', '055', '056', '058', '059', '06', '061', '063', '064', '065', '066', '067', '068', '069', '07', '073', '075', '076', '08', '080', '081', '082', '083', '084', '085', '09', '091', '093', '096', '099', '10', '100', '1000', '10036', '100th', '101', '102', '103', '1040', '104th', '105', '10573', '106', '106th', '107', '1074', '108', '109', '109th', '10th', '11', '110', '1100', '111', '112', '113', '1130', '114', '115', '116', '116th', '117', '118', '119', '11th', '12', '120', '121', '122', '123', '124', '125', '126', '128', '129', '12mo', '12th', '12wk', '12wks', '13', '130', '1307', '131', '132', '133', '134', '135', '136', '137', '138', '

In [11]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
X_transformed = tfidf_transformer.fit_transform(word_count_vector)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

In [13]:
import time

In [14]:
%%time
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report

est = LinearSVC(penalty='l1', dual=False, tol=1e-3)

# Calibrated with isotonic calibration
clf = CalibratedClassifierCV(est, cv=2, method='isotonic')

t0 = time.time()
clf.fit(X_train, y_train)
t1 = time.time()
y_pred = clf.predict(X_test)
t2 = time.time()
time_clf_train = t1-t0
time_clf_predict = t2-t1
print("Results for classifier")
print("Training time: %fs; Prediction time: %fs" % (time_clf_train, time_clf_predict))
print(classification_report(y_test, y_pred))

Results for classifier
Training time: 0.032964s; Prediction time: 0.000668s
              precision    recall  f1-score   support

    NEGATIVE       0.70      0.95      0.80       261
    POSITIVE       0.70      0.22      0.34       139

    accuracy                           0.70       400
   macro avg       0.70      0.59      0.57       400
weighted avg       0.70      0.70      0.64       400

CPU times: user 58.9 ms, sys: 12.5 ms, total: 71.4 ms
Wall time: 126 ms
